，一篇文章可以被简单地看作是一串单词序列，甚至是一串字符序列。 本节中，我们将解析文本的常见预处理步骤。 这些步骤通常包括：
+ 将文本作为字符串加载到内存中
+ 将字符串拆分成词元（如单词和字符）
+ 建立一个词表，将拆分的词元映射到数字索引
+ 将文本映射成数字索引序列，方便模型操作

In [1]:
import collections
import re
from d2l import torch as d2l

In [2]:
#@save
d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt',
                                '090b5e7e70c295757f55df93cb0a180b9691891a')

def read_time_machine():  #@save
    """将时间机器数据集加载到文本行的列表中"""
    with open(d2l.download('time_machine'), 'r') as f:
        lines = f.readlines()
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines]

lines = read_time_machine()
print(f'# 文本总行数: {len(lines)}')
print(lines[0])
print(lines[10])

# 文本总行数: 3221
the time machine by h g wells
twinkled and his usually pale face was flushed and animated the


### 8.2.2. 词元化
下面的tokenize函数把文本行列表作为输入
列表中每一个元素是一个文本序列（如一个文本行）
每个文本行又被拆分为一个词元列表，词元（token）是文本的基本单位
最后，返回一个由词元组成的列表，其中每个词元都是一个字符串string

In [3]:
def tokenize(lines, token='word'):
    '''把文本行拆分为单词或字符词元'''
    if token == 'word':
        return [line.split() for line in lines]
    elif token == 'char':
        return [list(line) for line in lines]
    else:
        print('错位，未知词元类型：',token)

In [7]:
tokens = tokenize(lines, token='word')
for i in range(11):
    # 前10行
    print(tokens[i])
# tokens是一个列表，里面含有多个列表，内部列表存放的元素是词元token

['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[]
[]
[]
[]
['i']
[]
[]
['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']


### 8.2.3. 词表
##### 列表中每个元素是一个文本序列（一行）
##### 每一行被拆分为词元列表， 词元就是token，也是文本的基本单位
词元的类型是字符串，而模型需要的输入是数字，因此这种类型不方便模型使用。
现在，让我们构建一个字典，通常也叫做词表（vocabulary）， 用来将字符串类型的词元映射到从 0 开始的数字索引中。
我们先将训练集中的所有文档合并在一起，
###### 对它们的唯一词元进行统计，得到的统计结果称之为语料（corpus）。
##### 然后根据每个唯一词元的出现频率，为其分配一个数字索引。 很少出现的词元通常被移除，这可以降低复杂性。
另外，语料库中不存在或已删除的任何词元都将映射到一个特定的未知词元“\<unk\>”。
我们可以选择增加一个列表，用于保存那些被保留的词元， 例如：填充词元（“\<pad\>”）； 序列开始词元（“\<bos\>”）； 序列结束词元（“\<eos\>”）。

In [34]:
class Vocab:
    '''文本词表'''
    def __init__(self,tokens=None, min_freq=0, reserved_tokens=None):
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []
        # 按出现频率排序——即得到语料Count({'word1':100,'word2},41)
        counter = count_corpus(tokens)
        # 得到频率后再根据值排序，降序，得到的是[('word2',5),('word1',4)]
        self._token_freqs = sorted(counter.items(), key=lambda x: x[1], reverse=True)

        # 根据索引得到token是一个列表，根据token得到索引就需要字典
        # 未知词元索引为0，放到最前面，可以根据idx得到token
        self.idx_to_token = ['<unk>'] + reserved_tokens
        # 根据token找到id，所以token是键，idx是值，idx的值需要到id_to_token中查找
        self.token_to_idx = {
            token: idx for idx, token in enumerate(self.idx_to_token)
        }
        # self.idx_to_token, self.token_to_idx = [], dict()
        for token, freq in self._token_freqs:
            if freq < min_freq:
                # token出现的频率小于最小频率
                break
            if token not in self.token_to_idx:
                # token不在字典中，就对列表和字典进行维护
                self.idx_to_token.append(token)
                self.token_to_idx[token] = len(self.idx_to_token)
    def __len__(self):
        return len(self.idx_to_token)

    def __getitem__(self, tokens):
        # tokens不是列表或者元组，就查找字典token_to_idx的idx
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        # 如果是字典，则对字典内继续遍历（char）
        return [self.__getitem__(token) for token in tokens]

    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]

    @property
    def unk(self):
        # 未知元素索引为0
        return 0
    @property
    def token_freqs(self):
        return self._token_freqs

def count_corpus(tokens):
    '''统计词元的频率'''
    # 这里的tokens可能是1d列表word，或者2d列表
    if len(tokens) == 0 or isinstance(tokens[0], list):
        # 把词元列表展开为一个列表
        tokens = [token for line in tokens for token in line]
    return collections.Counter(tokens)

In [35]:
# 测试
from collections import Counter
l1 = ['q','w','e','e','ee','a','a','a','b','b','b']
counter = Counter(l1)
print(counter)# 计算列表中每个元素的出现次数
# 虽然默认排序了，但是为了以防万一
s1 = sorted(counter.items(), key=lambda x:x[1],reverse=True)
print(s1)
print(counter.get('a'))
print(counter.get('aaaa', '0'))# 如果没有查找到aaaa就会了0


Counter({'a': 3, 'b': 3, 'e': 2, 'q': 1, 'w': 1, 'ee': 1})
[('a', 3), ('b', 3), ('e', 2), ('q', 1), ('w', 1), ('ee', 1)]
3
0


In [36]:
vocab = Vocab(tokens)
print(list(vocab.token_to_idx.items())[:10])

[('<unk>', 0), ('the', 2), ('i', 3), ('and', 4), ('of', 5), ('a', 6), ('to', 7), ('was', 8), ('in', 9), ('that', 10)]


把每一条文本转化为数字索引列表

In [37]:
for i in [0, 10]:
    print('文本：',tokens[i])
    print('索引：', vocab[tokens[i]])

文本： ['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
索引： [2, 20, 51, 41, 2184, 2185, 401]
文本： ['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']
索引： [2187, 4, 26, 1045, 363, 114, 8, 1422, 4, 1046, 2]


### 8.2.4. 整合所有功能
在使用上述函数时，我们将所有功能打包到load_corpus_time_machine函数中， 该函数返回corpus（词元索引列表）和vocab（时光机器语料库的词表）。 我们在这里所做的改变是：

为了简化后面章节中的训练，我们使用字符（而不是单词）实现文本词元化；

时光机器数据集中的每个文本行不一定是一个句子或一个段落，还可能是一个单词，因此返回的corpus仅处理为单个列表，而不是使用多词元列表构成的一个列表。

In [38]:
def load_corpus_time_machine(max_tokens=-1):  #@save
    """返回时光机器数据集的词元索引列表和词表"""
    lines = read_time_machine()
    tokens = tokenize(lines, 'char')
    vocab = Vocab(tokens)
    # 因为时光机器数据集中的每个文本行不一定是一个句子或一个段落，
    # 所以将所有文本行展平到一个列表中
    corpus = [vocab[token] for line in tokens for token in line]
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
    return corpus, vocab

corpus, vocab = load_corpus_time_machine()
len(corpus), len(vocab)

(170580, 28)